In [1]:
from eppy import modeleditor
from eppy.modeleditor import IDF
from matplotlib import pyplot as plt
import os
import pandas as pd
from besos import eppy_funcs as ef
from besos import sampling
from besos.evaluator import EvaluatorEP
from besos.parameters import RangeParameter, FieldSelector, FilterSelector, Parameter, expand_plist, wwr, CategoryParameter, GenericSelector
from besos.problem import EPProblem
import numpy as np

Define the .IDF, .IDFF, and Weather File (.EPW)

In [2]:
idd_file='/usr/local/EnergyPlus-9-0-1/Energy+.idd'
weather_file ='/home/ict4bd/Residential_oslo/weather_data/NOR_Oslo.Fornebu.014880_IWEC.epw'
building = ef.get_building('/home/ict4bd/Residential_oslo/IDF_Files/on3.idf')

In [3]:
zoneinfo = building.idfobjects["Zone"]
print(zoneinfo)

[
Zone,
    Blocco1:Zona3,            !- Name
    0,                        !- Direction of Relative North
    0,                        !- X Origin
    0,                        !- Y Origin
    0,                        !- Z Origin
    1,                        !- Type
    1,                        !- Multiplier
    ,                         !- Ceiling Height
    77.2899,                  !- Volume
    22.7323,                  !- Floor Area
    TARP,                     !- Zone Inside Convection Algorithm
    ,                         !- Zone Outside Convection Algorithm
    Yes;                      !- Part of Total Floor Area
, 
Zone,
    Blocco1:Zona1,            !- Name
    0,                        !- Direction of Relative North
    0,                        !- X Origin
    0,                        !- Y Origin
    0,                        !- Z Origin
    1,                        !- Type
    1,                        !- Multiplier
    ,                         !- Ceiling Heigh

Select the component of the building that will be changed during the simulation,
in particular:
- window to wall ratio;
- insulation thickness;
- ACH;
- orientation.

ACH is expressed as a number from 0 to 10 with steps of 1. In order to change it in the .IDF file, it must be converted as Air Flow Rate per zone [m^3/s]. 
?But first we isolate the infiltration for each zone that will be subtracted to the global flow rate?

In [4]:
#Ask to the prof if it makes sense
ZoneInfiltration =  building.idfobjects['ZoneInfiltration:DesignFlowRate']
for item in ZoneInfiltration:
    if item.Name == "Blocco1:Zona1 Infiltration":
        zone_1_infiltration = item.Design_Flow_Rate
    if item.Name == "Blocco1:Zona2 Infiltration":
        zone_2_infiltration = item.Design_Flow_Rate
    if item.Name == "Blocco1:Zona3 Infiltration":
        zone_3_infiltration = item.Design_Flow_Rate

Now we are able to calculate the Zone Flow Rate by proportioning the global flow rate to each room's surface

In [5]:
ACH = np.arange(1,11,1)
Flow_rate_1 = []
Flow_rate_2 = []
Flow_rate_3 = []

volume_building = 0
for Zone in building.idfobjects["Zone"]:
    volume_building += float(Zone.checkrange("Volume"))
    if (Zone.Name == 'Blocco1:Zona1'):
        volume_zone_1 = Zone.Volume
    if (Zone.Name == 'Blocco1:Zona2'):
        volume_zone_2 = Zone.Volume
    if (Zone.Name == 'Blocco1:Zona3'):
        volume_zone_3 = Zone.Volume
        
for i in ACH:
    global_Flow_rate = (i * volume_building)/3600
    Flow_rate_1.append((global_Flow_rate*volume_zone_1/volume_building))
    Flow_rate_2.append((global_Flow_rate*volume_zone_2/volume_building))
    Flow_rate_3.append((global_Flow_rate*volume_zone_3/volume_building))



Define the parameter that have to be changed during the simulation

In [6]:
insulation = FieldSelector(class_name = 'material', object_name='XPS Extruded Polystyrene  - CO2 Blowing_.35', field_name='Thickness')
insulationPR = Parameter(selector=insulation, value_descriptor=RangeParameter(0.0001, 0.35),
                         name='Insulation Thickness')

window_to_wall = wwr(CategoryParameter(options=[.05,.15,.5]))

orientation = FieldSelector(class_name='Building', field_name='North Axis')
orientationPR = Parameter(selector=orientation, value_descriptor=CategoryParameter(options=[0,90,180,270]),
                          name='Orientation')

zone_1_ventilation = FieldSelector(class_name = 'ZoneVentilation:DesignFlowRate', object_name = 'Blocco1:Zona1 Nat Vent', field_name = 'Design Flow Rate')
zone_1_ventilationPR = Parameter(selector = zone_1_ventilation, value_descriptor=RangeParameter(0.0001,10), name ="Zone 1 Vent")

zone_2_ventilation = FieldSelector(class_name = 'ZoneVentilation:DesignFlowRate', object_name = 'Blocco1:Zona2 Nat Vent', field_name = 'Design Flow Rate')
zone_2_ventilationPR = Parameter(selector = zone_2_ventilation, value_descriptor=RangeParameter(0.0001,10), name ="Zone 2 Vent")

zone_3_ventilation = FieldSelector(class_name = 'ZoneVentilation:DesignFlowRate', object_name = 'Blocco1:Zona3 Nat Vent', field_name = 'Design Flow Rate')
zone_3_ventilationPR = Parameter(selector = zone_3_ventilation, value_descriptor=RangeParameter(0.0001,10), name ="Zone 3 Vent")

parameters = [orientationPR, window_to_wall, insulationPR, zone_1_ventilationPR, zone_2_ventilationPR, zone_3_ventilationPR]

/home/ict4bd/py3/lib/python3.8/site-packages/besos/parameters.py:592: UserWarning: wwr is intended to be used with RangeParameter. Your value_descriptor is CategoryParameter(options=[0.05, 0.15, 0.5])
  warnings.warn(


Create a dictionary of samples that will be used during the simulation (i.e. the list of all the possible
combination of the previous parameter)

In [7]:
samples_temp =[]
wwr_values = [.05,.15,.5] #per velocizzare tolgo .5 e 0.05
thickness_values = np.linspace(0.001,0.35,5)
orientation_values = [0,90,180,270] #per velocizzare simulazione, tolgo due valori (270,360)

for orientation in orientation_values:
    for wwr in wwr_values:
        for thickness in thickness_values:
            for i in range(len(Flow_rate_1)):
                samples_temp.append({
                'Orientation': orientation,
                'Window to Wall Ratio': wwr,
                'Insulation Thickness': thickness,
                'Zone 1 Vent': Flow_rate_1[i],
                'Zone 2 Vent': Flow_rate_2[i],
                'Zone 3 Vent': Flow_rate_3[i]})

samples = pd.DataFrame.from_dict(samples_temp) #convert the dictionary to dataframe
samples #print the samples dataframe

,Orientation,Window to Wall Ratio,Insulation Thickness,Zone 1 Vent,Zone 2 Vent,Zone 3 Vent
0,0,0.05,0.001,0.004122,0.013850,0.021469
1,0,0.05,0.001,0.008245,0.027701,0.042939
2,0,0.05,0.001,0.012367,0.041551,0.064408
3,0,0.05,0.001,0.016489,0.055401,0.085878
4,0,0.05,0.001,0.020611,0.069252,0.107347
...,...,...,...,...,...,...
595,270,0.50,0.350,0.024734,0.083102,0.128817
596,270,0.50,0.350,0.028856,0.096952,0.150286
597,270,0.50,0.350,0.032978,0.110802,0.171755
598,270,0.50,0.350,0.037100,0.124653,0.193225


Define the objective of the evaluator and create a dataframe of the outputs

In [8]:
objectives = ['Electricity:Facility','DistrictCooling:Facility','DistrictHeating:Facility']

problem = EPProblem(parameters, objectives)
evaluator = EvaluatorEP(problem, building, out_dir='outputdirSimulation', error_mode='Failfast', err_dir='outputdirSimulation',
                        epw = weather_file)
outputs = evaluator.df_apply(samples, keep_input=True)  

KeyboardInterrupt: 

Evaluate the total area of the building so that we can find the consumptione over square meter.

In [9]:
total_area= 0
for Zones in building.idfobjects["Zone"]:
    total_area += float(Zones.checkrange("Floor_Area")) 
    print(Zones.checkrange("Floor_Area"))
total_area

22.7323
4.3647
14.665


41.762

Transform the output from [J] to [kWh/m2]

In [ ]:
outputs['Electricity:Facility'] /= (3.6e6 * total_area) #kWh/m2
outputs['DistrictCooling:Facility'] /= (3.6e6 * total_area)
outputs['DistrictHeating:Facility'] /= (3.6e6 * total_area)
outputs['TotalConsumption'] = outputs['Electricity:Facility'] + outputs['DistrictCooling:Facility'] + outputs['DistrictHeating:Facility']
print(outputs)

Find the index of minimum value of total consumption and print the corresponding parameter values.

In [ ]:
min_value_index = outputs[['TotalConsumption']].idxmin()
print(outputs.iloc[min_value_index])